In [6]:
import geojson
import json
import os
import sys
import requests
import time
import datetime
import pytz
import logging
import logging.handlers
import argparse
import re
import math
import random
import traceback
from collections import defaultdict
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings 
warnings.filterwarnings('ignore')

In [8]:
# URLs for the data
STATIONS_URL = "https://api.energyandcleanair.org/stations?country=GB,US,TR,PH,IN,TH&format=geojson"
COUNTRIES_URL = "https://datahub.io/core/geo-countries/r/countries.geojson"

def download_and_save_data():
    """Download and save GeoJSON data from APIs"""
    # Download stations data
    try:
        response = requests.get(STATIONS_URL)
        response.raise_for_status()
        stations_data = response.json()
        with open('stations.geojson', 'w') as f:
            json.dump(stations_data, f)
    except Exception as e:
        print(f"Error downloading stations data: {e}")
        return False

    # Download countries data
    try:
        response = requests.get(COUNTRIES_URL)
        response.raise_for_status()
        countries_data = response.json()
        with open('countries.geojson', 'w') as f:
            json.dump(countries_data, f)
    except Exception as e:
        print(f"Error downloading countries data: {e}")
        return False
    
    return True

def load_data():
    """Load GeoJSON data from local files or download if not present"""
    if not (os.path.exists('stations.geojson') and os.path.exists('countries.geojson')):
        if not download_and_save_data():
            raise Exception("Failed to download required data")
    
    with open('stations.geojson', 'r') as f:
        stations_data = json.load(f)
    with open('countries.geojson', 'r') as f:
        countries_data = json.load(f)
    
    return stations_data, countries_data

def get_country_area(country_code, countries_data):
    """Calculate country area in square kilometers"""
    for feature in countries_data['features']:
        if feature['properties']['ISO_A2'] == country_code:
            # Using GeoPandas would be more accurate, but for this example we'll use
            # the provided AREA property or calculate a rough approximation
            if 'AREA' in feature['properties']:
                return feature['properties']['AREA']
            else:
                # Rough approximation if AREA is not provided
                return 1000000  # placeholder value
    return None

def is_point_in_polygon(point, geometry):
    """Check if a point is inside a polygon or multipolygon"""
    if geometry['type'] == 'MultiPolygon':
        return any(is_point_in_single_polygon(point, polygon[0]) 
                  for polygon in geometry['coordinates'])
    elif geometry['type'] == 'Polygon':
        return is_point_in_single_polygon(point, geometry['coordinates'][0])
    return False

def is_point_in_single_polygon(point, polygon):
    """Ray casting algorithm for point-in-polygon test"""
    x, y = point
    n = len(polygon)
    inside = False
    p1x, p1y = polygon[0]
    for i in range(n):
        p2x, p2y = polygon[i]
        if y > min(p1y, p2y):
            if y <= max(p1y, p2y):
                if x <= max(p1x, p2x):
                    if p1y != p2y:
                        xinters = (y-p1y)*(p2x-p1x)/(p2y-p1y)+p1x
                    if p1x == p2x or x <= xinters:
                        inside = not inside
        p1x, p1y = p2x, p2y
    return inside

def count_stations_in_country(stations_data, country_geometry):
    """Count number of stations within a country's boundaries"""
    count = 0
    for feature in stations_data['features']:
        station_coords = feature['geometry']['coordinates']
        if is_point_in_polygon(station_coords, country_geometry):
            # Check if station measures PM10
            if 'parameters' in feature['properties']:
                parameters = feature['properties']['parameters']
                if isinstance(parameters, list) and 'pm10' in parameters:
                    count += 1
    return count

def calculate_station_densities():
    """Calculate and display station densities for specified countries"""
    # Load data
    stations_data, countries_data = load_data()
    
    # Countries to analyze
    country_codes = {'US': 'United States', 'GB': 'United Kingdom', 
                    'TR': 'Turkey', 'TH': 'Thailand', 
                    'PH': 'Philippines', 'IN': 'India'}
    
    results = []
    for code, name in country_codes.items():
        # Get country geometry and area
        country_geom = None
        for feature in countries_data['features']:
            if feature['properties']['ISO_A2'] == code:
                country_geom = feature['geometry']
                break
        
        if country_geom is None:
            print(f"Could not find geometry for {name}")
            continue
            
        area = get_country_area(code, countries_data)
        if area is None:
            print(f"Could not find area for {name}")
            continue
            
        # Count PM10 stations
        num_stations = count_stations_in_country(stations_data, country_geom)
        
        # Calculate density per 1000 sq km
        density = (num_stations / area) * 1000
        
        results.append({
            'Country': name,
            'PM10 Stations': num_stations,
            'Area (sq km)': area,
            'Density (per 1000 sq km)': density
        })
    
    # Create DataFrame and sort by density
    df = pd.DataFrame(results)
    df = df.sort_values('Density (per 1000 sq km)', ascending=False)
    
    # Display results
    pd.set_option('display.float_format', '{:.4f}'.format)
    print("\nStation Density Analysis Results:")
    print(df.to_string(index=False))
    
    return df

if __name__ == "__main__":
    calculate_station_densities()


Station Density Analysis Results:
       Country  PM10 Stations  Area (sq km)  Density (per 1000 sq km)
 United States              0       1000000                    0.0000
United Kingdom              0       1000000                    0.0000
        Turkey              0       1000000                    0.0000
      Thailand              0       1000000                    0.0000
   Philippines              0       1000000                    0.0000
         India              0       1000000                    0.0000
